In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hgmd = pd.read_csv("/data/projects/processBio/hgmd/variant_df.csv")

In [ ]:
hgmd = hgmd.rename({"GENE":"GeneSymbol"},axis=1)

In [ ]:
hgmd = hgmd.assign(label=True)

In [ ]:
hgmdX = np.load("/data/projects/processBio/hgmd/X.npy")

In [ ]:
clinvarDF = pd.read_csv("/data/projects/processBio/clinvar/clinvar/parseClinvarWithVariant.csv").rename({"variantSeq": "seq"},axis=1)

In [ ]:
clinvarDF[clinvarDF.seq != ""]

In [ ]:
import pickle
clinvarEmbeddings = pickle.load(open("/data/projects/processBio/clinvar/clinvar/parseClinvarRepresentations.pkl","rb"))

In [ ]:
clinvarDF = clinvarDF[[len(e) > 0 for e in clinvarEmbeddings]]

In [ ]:
clinvarDF

In [ ]:
clinvarX = np.load("/data/projects/processBio/clinvar/clinvar/X.npy")

In [ ]:
clinvarX.shape

In [ ]:
gnomad = pd.read_pickle("/data/projects/processBio/gnomad/gnomad.missenseVariants.pd.pkl")

In [ ]:
gnomad = gnomad[gnomad.apply(lambda row: len(row.seq) > int(row.Protein_position) - 1,axis=1)]

In [ ]:
gnomadX = np.load("/data/projects/processBio/gnomad/gnomadValidMissenseVariantEmbeddings.npy")

In [ ]:
gnomadX.shape

In [ ]:
gnomadSymbols = gnomad.INFO.apply(lambda i: i["SYMBOL"] if "SYMBOL" in i else "")
gnomad=gnomad.assign(GeneSymbol=gnomadSymbols)

In [ ]:
gnomad[gnomad.GeneSymbol == ""]

In [ ]:
clinvarSymbols = clinvarDF["GeneSymbol"]

In [ ]:
hgmdSymbols = hgmd["GeneSymbol"]

In [ ]:
symbols = pd.concat([clinvarSymbols,
                     hgmdSymbols,
                     gnomadSymbols])

In [ ]:
symbols.shape

In [ ]:
symbols.value_counts().shape, "{:.3f}".format(symbols.value_counts().mean())

# Combine all datasets

In [ ]:
gnomad = gnomad.assign(label=np.nan)

In [ ]:
data =pd.concat((gnomad[["seq","label","GeneSymbol"]],
           clinvarDF[["seq","label","GeneSymbol"]],
           hgmd[["label", "seq","GeneSymbol"]]))

In [ ]:
X = np.concatenate((gnomadX, clinvarX,hgmdX))

In [ ]:
gnomadX.shape, clinvarX.shape, hgmdX.shape, X.shape

In [ ]:
clinvarDF.label.value_counts()

In [ ]:
np.save("/data/projects/processBio/X.npy",X)

In [ ]:
data.to_pickle("/data/projects/processBio/df.pd.pkl")

In [ ]:
alphas = []
numPos = []
numNeg = []
for (idx,group) in data[~data.label.isna()].groupby("GeneSymbol"):
    labels = group.label
    if labels.sum() > 0 and labels.sum() < labels.shape[0]:
        alphas.append(labels.sum() / labels.shape[0])
        numPos.append(labels.sum())
        numNeg.append(labels.shape[0] - labels.sum())

In [ ]:
len(alphas)

In [ ]:
(data.GeneSymbol == "").any()

In [ ]:
data.groupby("GeneSymbol").count()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(alphas,)

# Can I skip clustering by using the mutpred data clusters?

In [ ]:
mutpreddf = pd.read_csv("/data/projects/mutpred/mutpred2_revised_val_cleaned_cluster50.txt",header=10,delimiter="\t")
mutpreddf = mutpreddf.assign(symbol=mutpreddf.internal_id.apply(lambda x: [xi.replace("_HUMAN","") for xi in x.split(";")  if "HUMAN" in xi]))

In [ ]:
mutpreddf

In [ ]:
mpsymbols = set([s[0] for s in mutpreddf.symbol if len(s)])

In [ ]:
pd.Series(data.GeneSymbol.unique()).isin(mpsymbols).value_counts()

I cannot...

In [ ]:
data[~data.label.isna()].groupby("GeneSymbol").count().mean()

In [ ]:
data[~data.label.isna()]

In [ ]:
data.label.value_counts()

In [ ]:
data.label.shape